In [291]:
from bayes_opt import BayesianOptimization
import matplotlib.pyplot as plt
from sklearn import datasets
import numpy as np
from sklearn.preprocessing import Normalizer, MaxAbsScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from datetime import datetime
import pickle

In [148]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [149]:
%run SVR_MAPE_Library.ipynb

In [305]:
y1 =pd.read_excel(r'Data//Data1.xlsx').set_index("fecha").loc["2007-01-01":"2020-03-30"]
X1 = pd.read_csv("X.csv")

X = X1.drop(["fecha"], axis = 1).to_numpy(); y = y1.to_numpy()

# X, X_test, y, y_test = train_test_split(X, y, test_size = 7, shuffle = False)

# scaler = MaxAbsScaler(); scaler.fit(X); X = scaler.transform(X)
# X_test = scaler.transform(X_test)

# scaler1 = MaxAbsScaler(); scaler1.fit(y); y = scaler1.transform(y).reshape(-1)

In [306]:
def bas_optit(C, epsilon, gamma, mu, lmbda): 
#   n: test size, itr: # of iterations
    n = 7; itr = 1
    
    cas = {}; mape = []; it = itr+1
    yl = len(y)
    
    # parameters
    hyperparameters = {
        'kernel' : "rbf",
        'C' : C, 
        'epsilon' : epsilon, 
        'mu' : mu,
        'lmbda' : lmbda,
        'gamma' : gamma, 
    }
    cas[0] = hyperparameters
    
    itera = np.flip(np.arange(1, it))
    for i in itera:
        j = i*n
        
        # y, X split
        X_train = X[:yl - j, :]; X_test = X[yl - j : yl - (j-n), :]
        y_train = y[:yl - j];    y_test = y[yl - j : yl - (j-n)]
        
        # test index
        y_idx = y1.index[yl - j : yl - (j-n)]
        
        # rescale X and y
        scaler = MaxAbsScaler(); scaler.fit(X_train); 
        X_train = scaler.transform(X_train); X_test = scaler.transform(X_test)
        
        scaler1 = MaxAbsScaler(); scaler1.fit(y_train)
        y_train = scaler1.transform(y_train).reshape(-1)
        y_test = y_test.reshape(-1)
        

        # fit and predict
        model = SVR_general_cvxopt(**hyperparameters).fit(X_train, y_train)
        pred = model.predict(X_test)
        
        # rescale y_test
        y_pred = scaler1.inverse_transform(pred.reshape(-1, 1))
        
        # keep data
        yi = pd.DataFrame(y_test, index = y_idx, columns = ["real"])
        yi["predict"] = y_pred.reshape(-1)
        yi["resta"] = yi.real - yi.predict
        yi["error"] = (np.abs((yi.real - yi.predict)/yi.real))
        cas[i] = y_test
        mape.append(yi.error.mean()*100)
    
    # Store prediction
    name_ = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")    
    with open(f'{name_}', 'wb') as to_write:
        pickle.dump(cas, to_write)
    
    mape_mean = sum(mape)/it
    
    return -mape_mean

In [307]:
# def bas_opt(C, epsilon, gamma, mu, lmbda):
#     predict = SVR_general_cvxopt(kernel = "rbf", C = C, epsilon = epsilon, mu = mu, lmbda = lmbda, gamma = gamma).fit(X, y).predict(X_test)
#     y_pred = scaler1.inverse_transform(predict.reshape(-1, 1))
    
#     return -np.mean(np.abs((y_pred - y_test)/y_test))*100

In [308]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [309]:
# Bounded region of parameter space
pbounds = {'C': (0.1, 10), 'epsilon': (0.01, 10), 'gamma': (0.001, 0.1), 'mu': (0.01, 5), 'lmbda': (0.01, 5)}

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=bas_optit, pbounds=pbounds, random_state=1, verbose=2)

# keep data
logger = newJSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [310]:
# optimize
optimizer.maximize(init_points=0, n_iter=1)

In [71]:
len(optimizer.res)

30

In [72]:
for i in range(len(optimizer.res)):
    print(f"{i} {optimizer.res[i]['target']:.2f}")

0 -1.84
1 -3.44
2 -3.79
3 -5.15
4 -4.98
5 -2.25
6 -4.15
7 -3.85
8 -5.00
9 -5.07
10 -2.20
11 -2.33
12 -4.97
13 -87.28
14 -3.67
15 -16.80
16 -5.43
17 -144.67
18 -11.16
19 -2.30
20 -2.11
21 -3.16
22 -2.24
23 -1.89
24 -2.35
25 -2.12
26 -2.23
27 -5.18
28 -1.85
29 -1.84


In [73]:
optimizer.res[0]

{'target': -1.8416600603973563,
 'params': {'C': 4.228517846555483,
  'epsilon': 7.20604168948716,
  'gamma': 0.0010113231069171437,
  'lmbda': 1.5186395374328805,
  'mu': 0.7423118951773942}}

In [ ]:
with open('07_04_2021_01_13_55','rb') as read_file:
    df = pickle.load(read_file)